In [1]:
from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/기계시스템설계/23년/baseline/baseline

/content/drive/MyDrive/Colab Notebooks/기계시스템설계/23년/baseline/baseline


In [4]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


SystemError: ignored

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/기계시스템설계/23년/baseline/baseline'

In [6]:
!pip install fmpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 13.5 MB/s eta 0:00:00


In [7]:
from fmpy import read_model_description, extract
from fmpy.fmi2 import FMU2Slave
from fmpy.util import plot_result, download_test_file
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt

import random
import numpy as np
import torch
import argparse
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras

from collections import deque
import numpy as np
from tqdm import tqdm

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import scipy.signal
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.normal import Normal
from torch import FloatTensor as FT

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp

In [8]:
!pip install -q -U tensorflow>=2.9.0
import tensorflow as tf

In [68]:
fmu_filename = 'HEV_TMED_Simulator_Rearrange_230213_linux.fmu'
start_time = 0.0
stop_time = 1800.0
step_size = 0.01
soc_init = 67

# WLTP profile에 대하여 시험
with open("wltp_vehicle_speed_profile_real.csv") as file_name:
    vehicle_speed_profile = np.loadtxt(file_name, delimiter=",")

In [69]:
# init #
vrs = {}
state = np.array([soc_init, 0, 0, 0, 0, 0, 0, 0, 0]).reshape(1,-1)
action_upper_bound = 15000
action_lower_bound = -15000
#self.action_space = [[13500, 2000], [500, 2000], [13500, -15000], [500, -15000]]
obssize = len(state[0])
actsize = 1

In [70]:
# reset #
model_description = read_model_description(fmu_filename)
for variable in model_description.modelVariables:
  vrs[variable.name] = variable.valueReference
unzipdir = extract(fmu_filename)
fmu = FMU2Slave(guid=model_description.guid,
          unzipDirectory=unzipdir,
          modelIdentifier=model_description.coSimulation.modelIdentifier,
          instanceName='instance1')
fmu.instantiate()
fmu.setupExperiment(startTime=start_time)
fmu.enterInitializationMode()
fmu.exitInitializationMode()
        
vr_input1 = vrs['Driver_sVeh_Target_kph']
vr_input2 = vrs['Engine_on_line']
vr_input3 = vrs['Engine_off_line']
vr_output1 = vrs['TgMod_fPt']
vr_output2 = vrs['P2_wElec_W']
vr_output3 = vrs['P4_wElec_W']
vr_output4 = vrs['ObP2_wElecBIntv_W']
vr_output5 = vrs['EV_on_line']
vr_output6 = vrs['EV_off_line']
vr_output7 = vrs['Bat_SOC']
vr_output8 = vrs['PT_tqTmInDmd_Nm_P2']
vr_output9 = vrs['P0_wElec_W']
vr_output10 = vrs['Pwr_Aux_W']
vr_output11 = vrs['ObEng_nEng_Rpm']
vr_output12 = vrs['TrEtp_tqEngMAF_Nm']
vr_output13 = vrs['rpm_P0']
vr_output14 = vrs['TM_F_nTmIn_rpm']
vr_output15 = vrs['TM_R_nTmIn_rpm']
vr_output16 = vrs['eBat_kWh']
vr_output17 = vrs['nEngOn']
vr_output18 = vrs['TrP0_tqP0_Nm']
vr_output19 = vrs['TrP2_tqP2_Nm']
vr_output20 = vrs['TrP4_tqP4_Nm']
vr_output21 = vrs['TCU_F_fCurGe']
vr_output22 = vrs['TCU_F_fTarGe']
vr_output23 = vrs['Driver_sVeh_kph']
vr_output24 = vrs['Eng_eff_avg']
vr_output25 = vrs['TM_F_P0P2_eff_avg']
vr_output26 = vrs['TM_F_P0P4_eff_avg']
vr_output27 = vrs['BSFC_g_kWh[1]']
vr_output28 = vrs['BSFC_g_kWh[2]']
vr_output29 = vrs['BSFC_g_kWh[3]']

In [71]:
# step #
# time 을 하나씩 increment 하기
actionspace = np.linspace(-1.0,1.0,101)
policy = []

for j in range(10):
  time = start_time + step_size * j
  instant_veh_speed = np.interp(time, vehicle_speed_profile[:,0], vehicle_speed_profile[:,1])
  reward_std = -np.inf
  action_list = []
  fmu.doStep(currentCommunicationPoint=time, communicationStepSize=step_size)
  for i in range(len(actionspace)):
    action = actionspace[i] * action_upper_bound
    fmu.setReal([vr_input1, vr_input2, vr_input3], [instant_veh_speed, action, action])
    state_fmu = fmu.getReal(np.arange(30))
    state_column = np.array([vrs['Bat_SOC'], vrs['nEngOn'], vrs['PT_tqTmInDmd_Nm_P2'], vrs['ObEng_nEng_Rpm'], vrs['TrEtp_tqEngMAF_Nm'], vrs['TrP0_tqP0_Nm'], vrs['TrP2_tqP2_Nm'], vrs['Driver_sVeh_kph'], vrs['BSFC_g_kWh[1]']])
    state = np.array(state_fmu)[state_column]
    soc = state_fmu[vrs['Bat_SOC']]
    BSFC = state_fmu[vrs['BSFC_g_kWh[1]']]
    reward = - 0.1*(soc_init - soc) ** 2 - BSFC # 연료 사용량
    print('reward : {0}, state : {1}'.format(reward,state))
    if reward_std < reward:
      action_list = [actionspace[i]]
      # print('a')
      reward_std = reward
    elif reward_std == reward:
      action_list.append(actionspace[i])
      # print('b')
    
    if i == len(actionspace)-1:
      print(action_list)
      policy.append(actionspace[np.argmin(np.abs(action_list))])
      print('timestep : {}'.format(j))
      print('policy : {}'.format(policy[j]))
      print('-----------------------------------------------------------------------------------------------------')
      




    # is_done = lambda time: time >= stop_time
    
    # time += step_size
    #     # return self.state.reshape(1,-1), reward, is_done(self.time), None
        # return self.fmu


reward : -4.1000932710341396e-10, state : [66.99993597  0.          0.          0.          0.          0.
  0.          0.          0.        ]
reward : -4.1000932710341396e-10, state : [66.99993597  0.          0.          0.          0.          0.
  0.          0.          0.        ]
reward : -4.1000932710341396e-10, state : [66.99993597  0.          0.          0.          0.          0.
  0.          0.          0.        ]
reward : -4.1000932710341396e-10, state : [66.99993597  0.          0.          0.          0.          0.
  0.          0.          0.        ]
reward : -4.1000932710341396e-10, state : [66.99993597  0.          0.          0.          0.          0.
  0.          0.          0.        ]
reward : -4.1000932710341396e-10, state : [66.99993597  0.          0.          0.          0.          0.
  0.          0.          0.        ]
reward : -4.1000932710341396e-10, state : [66.99993597  0.          0.          0.          0.          0.
  0.          0.       

In [34]:
len(actionspace)

21

In [47]:
time
start_time

0.0

In [ ]:
fmu.setReal